In [1]:
#default_exp ghtop

# ghtop

> I cannot believe this library name is not already taken on pypi.

In [2]:
#export
import time, sys, signal, shutil, os, json, enlighten, emoji, blessed
from ghtop.dashing import *
from collections import defaultdict

from fastcore.utils import *
from fastcore.foundation import *
from fastcore.script import *
from ghapi.all import *

In [3]:
#export
term = Terminal()
logfile = Path("log.txt")

In [4]:
#export
def github_auth_device():
    auth = GhDeviceAuth()
    wb = input(f"""First copy your one-time code: {term.yellow}{auth.user_code}{term.normal}
    Then visit {auth.verification_uri} in your browser, and paste the code when prompted.
    Shall we try to open the link for you? [y/n] """)
    if wb[0].lower()=='y': auth.open_browser()

    print ("Waiting for authorization...", end='')
    token = auth.wait(lambda: print('.', end=''))
    print("Authenticated with GitHub!")
    return token

In [5]:
#export
def exit(msg):
    print(msg, file=sys.stderr)
    sys.exit()

In [6]:
#export
def get_token():
    path = Path.home()/".ghtop_token"
    if path.is_file():
        try: return path.read_text().strip()
        except: exit("Error reading token")

    token = github_auth_device()
    path.write_text(token)
    return token

In [7]:
token = get_token()

In [8]:
#export
def limit_cb(rem,quota):
    w='WARNING '*7
    if rem < 1000: print(f"{w}\nRemaining calls: {rem} out of {quota}\n{w}")

In [9]:
#export
api = GhApi(limit_cb=limit_cb)

In [10]:
#export
def fetch_events(): return api.activity.list_public_events(per_page=100)

def _maybeAttrDict(x): return AttrDict(x) if isinstance(x, dict) else x

In [11]:
#export
def read_json_log():
    try: res = logfile.read_json()
    except FileNotFoundError: return []    
    return L(res).map(dict2obj) if isinstance(res,list) else L([dict2obj(res)])

In [12]:
#export
Events = dict(
    IssuesEvent_closed=('⭐', 'closed', noop),
    IssuesEvent_opened=('📫', 'opened', noop),
    IssueCommentEvent=('💬', 'commented on', term.white),
    PullRequestEvent_opened=('✨', 'opened a pull request', term.yellow),
    PullRequestEvent_closed=('✔', 'closed a pull request', term.green),
)

In [13]:
#export
printed_event_ids = {}

In [14]:
#export
def to_log(e):
    login,repo,pay = e.actor.login,e.repo.name,e.payload
    typ = e.type + (f'_{pay.action}' if e.type in ('PullRequestEvent','IssuesEvent') else '')
    emoji,msg,color = Events.get(typ, [0]*3)
    if emoji:
        xtra = '' if e.type == "PullRequestEvent" else f' issue # {pay.issue.number}'
        d = try_attrs(pay, "pull_request", "issue")
        return color(f'{emoji} {login} {msg}{xtra} on repo {repo[:20]} ("{d.title[:50]}...")')
    elif e.type == "ReleaseEvent": return f'🚀 {login} released {e.payload.release.tag_name} of {repo}'

In [15]:
#export
def print_event(e, commits_counter):
    if e.id in printed_event_ids: return
    printed_event_ids[e.id] = 1
    login = e.actor.login
    if "bot" in login or "b0t" in login: return  # Don't print bot activity (there is a lot!)

    res = to_log(e)
    if res: print(res)
    elif e.type == "PushEvent": [commits_counter.update() for c in e.payload.commits]
    elif e.type == "SecurityAdvisoryEvent": print(term.blink("SECURITY ADVISORY"))

In [27]:
printed_event_ids={}
for e in fetch_events():
    if e.type in ('IssuesEvent','ReleaseEvent','PullRequestEvent'): print_event(e,None)

✔ AlonBru closed a pull request on repo suvelocity/challenge ("Documentation...")
📫 lupupitkanen opened issue # 7980 on repo hashicorp/terraform- ("Add support for transaction_log_retention_days in ...")
📫 amulyamalla opened issue # 202 on repo keycloak/keycloak-co ("Unable import user from LDAP ...")
✔ SSunSShine closed a pull request on repo SSunSShine/QAsystem ("新增对API的测试模块，顺带更新下README.md...")
✨ andipaetzold opened a pull request on repo contentful/ui-extens ("Run Cypress tests in parallel...")
✨ JoshMarler opened a pull request on repo nick-thompson/bluepr ("Fix broken hot-reload behaviour....")
✨ TaniaBondarenko opened a pull request on repo OleksiyRudenko/a-tin ("List a tiny JS world by TaniaBondarenko...")
✨ CreeperPlanet26 opened a pull request on repo Maxisthemoose/tycoon ("MainEmbed class...")
✔ TofikAliarov closed a pull request on repo Ihor-malyshko/Kidsli ("getting info from forms...")
✨ voryzen opened a pull request on repo VivumLab/VivumLab ("refactor(service): fixes pix

In [36]:
#export
def gen_events():
    "Generate an infinite stream of events"
    while True: yield from fetch_events()

In [37]:
#export    
def write_logs(events):
    Path("tmp.log").write_text(json.dumps(events, indent=2))
    shutil.move("tmp.log", logfile)

def tail_events():
    manager = enlighten.get_manager()
    commits = manager.counter(desc='Commits', unit='commits', color='green')
    for ev in genevents():
        combined = (g+read_json_log()).sorted(using_attr(int,"id"))
        write_logs(obj2dict(combined))
        for x in combined: print_event(x, commits)
        time.sleep(0.2)

In [18]:
#export
def watch_users():
    users,users_events = defaultdict(int),defaultdict(lambda: defaultdict(int))
    while True:
        for x in fetch_events():
            login = x.actor.login
            users[login] += 1
            users_events[login][x.type] += 1

        print (term.clear())
        print ("User".ljust(30), "Events".ljust(6), "PRs".ljust(5), "Issues".ljust(6), "Pushes".ljust(7))
        sorted_users = sorted(users.items(), key = lambda kv: (kv[1], kv[0]), reverse=True)
        for i in range(20):
            u = sorted_users[i]
            ue = users_events[u[0]]
            print(u[0].ljust(30), str(u[1]).ljust(6), 
                  str(ue.get('PullRequestEvent', '')).ljust(5), 
                  str(ue.get('IssuesEvent', '')).ljust(6), 
                  str(ue.get('PushEvent', '')).ljust(7))
        time.sleep(1)

In [19]:
#export
def push_to_log(e):
    return f"{e.actor.login} pushed {len(e.payload.commits)} commits to repo {e.repo.name}"

In [20]:
#export
def str_clean(s): return s[:95]

In [21]:
#export
def signal_handler(sig, frame):
    if sig != signal.SIGINT: return
    print(term.exit_fullscreen())
    print(term.clear())
    print(term.normal)
    sys.exit(0)

In [52]:
#export
def logwin(title,color): return Log(title=title,border_color=2,color=color)
def quad_logs():
    term.enter_fullscreen()
    ui = HSplit(VSplit(logwin('Issues', color=7),        logwin('Commits' , color=3)),
                VSplit(logwin('Pull Requests', color=4), logwin('Releases', color=5)))

    issues,commits,prs,releases = all_items = ui.items[0].items+ui.items[0].items
    for o in all_items: o.append(" ")

    d = dict(PushEvent=commits, IssuesEvent=issues, IssueCommentEvent=issues, PullRequestEvent=prs, ReleaseEvent=releases)
    for xs in chunked((o for o in gen_events() if o.type in d), 10):
        for x in xs:
            f = [to_log,push_to_log][x.type == 'PushEvent']
            d[x.type].append(str_clean(f(x)))
        ui.display()
        time.sleep(0.1)

In [23]:
#export
def simple():
    for ev in genevents(): print(f"{ev.actor.login} {ev.type} {ev.repo.name}")

In [25]:
#export
def _help(): exit("Usage: ghtop <tail|quad|users|simple>")
if __name__ == '__main__' and not IN_NOTEBOOK:
    if len(sys.argv) < 2: exit(help_msg)
    signal.signal(signal.SIGINT, signal_handler)
    dict(tail=tail_events, quad=quad_logs, users=watch_users, simple=simple
        ).get(sys.argv[1],help)()